In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchtext.datasets import SST
from torchtext.data import Field, BucketIterator

# 定义Fields
TEXT = Field(lower=True, fix_length=200, batch_first=True)
LABEL = Field(sequential=False)

# 加载SST数据集
train, valid, test = SST.splits(TEXT, LABEL)

# 建立词汇表并初始化词嵌入
TEXT.build_vocab(train, max_size=20000, min_freq=10)
LABEL.build_vocab(train)

In [2]:
# 定义模型
class TextClassificationModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_layers, num_classes):
        super(TextClassificationModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.rnn = nn.RNN(embedding_dim, hidden_dim, num_layers, batch_first=True)
        self.pooling = nn.AdaptiveAvgPool1d(1)
        self.classifier = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, num_classes)
        )

    def forward(self, x):
        embedded = self.embedding(x)
        rnn_output, _ = self.rnn(embedded)
        pooled = self.pooling(rnn_output.permute(0, 2, 1))
        pooled = pooled.view(pooled.size(0), -1)
        output = self.classifier(pooled)
        return output


In [3]:
# 创建模型、优化器和损失函数
model = TextClassificationModel(len(TEXT.vocab), 100, 128, 2, len(LABEL.vocab))
optimizer = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss()

In [4]:
# 创建数据迭代器
batch_size = 16
train_iter, valid_iter, test_iter = BucketIterator.splits((train, valid, test), batch_size=batch_size, repeat=False)

# 训练循环
def train_model(model, iterator, optimizer, criterion):
    model.train()
    for batch in iterator:
        optimizer.zero_grad()
        predictions = model(batch.text)
        loss = criterion(predictions, batch.label)
        loss.backward()
        optimizer.step()

In [5]:
# 验证循环
def evaluate_model(model, iterator, criterion):
    model.eval()
    total_loss = 0
    correct = 0
    with torch.no_grad():
        for batch in iterator:
            predictions = model(batch.text)
            loss = criterion(predictions, batch.label)
            total_loss += loss.item()
            predicted_labels = predictions.argmax(1)
            correct += (predicted_labels == batch.label).sum().item()
    return total_loss / len(iterator), correct / len(iterator.dataset)


In [6]:
# 训练模型
N_EPOCHS = 10
for epoch in range(N_EPOCHS):
    train_model(model, train_iter, optimizer, criterion)
    valid_loss, valid_acc = evaluate_model(model, valid_iter, criterion)
    print(f'Epoch: {epoch+1:02}')
    print(f'\tValidation Loss: {valid_loss:.3f} | Validation Acc: {valid_acc*100:.2f}%')

# 测试模型
test_loss, test_acc = evaluate_model(model, test_iter, criterion)
print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Epoch: 01
	Validation Loss: 1.067 | Validation Acc: 45.69%
Epoch: 02
	Validation Loss: 1.075 | Validation Acc: 38.87%
Epoch: 03
	Validation Loss: 1.058 | Validation Acc: 41.05%
Epoch: 04
	Validation Loss: 1.060 | Validation Acc: 38.78%
Epoch: 05
	Validation Loss: 1.064 | Validation Acc: 40.24%
Epoch: 06
	Validation Loss: 1.060 | Validation Acc: 40.24%
Epoch: 07
	Validation Loss: 1.059 | Validation Acc: 40.24%
Epoch: 08
	Validation Loss: 1.056 | Validation Acc: 42.87%
Epoch: 09
	Validation Loss: 1.059 | Validation Acc: 41.60%
Epoch: 10
	Validation Loss: 1.058 | Validation Acc: 42.96%
Test Loss: 1.034 | Test Acc: 44.84%
